In [1]:
import os,sys,subprocess,glob,cftime,importlib,pickle,itertools
from datetime import datetime
import xarray as xr
import numpy as np
sys.path.append('../')

sys.path.append('../../REA_with_CESM2')
from ensembles.ensemble_GKLT import ensemble_GKLT
from data_extractor.data_extractor import extract_rea,extract_initial

from experiment_configuration.experiment import experiment

%load_ext autoreload
%autoreload 2

In [4]:
data_dir = '/work/bb1152/u290372/REA_output/heat_wEU_JJA/NCAR/CESM2/'

In [39]:
# uniqueness
for experiment_identifier in [f"c{i}" for i in range(1,6)] + [f"p{i}" for i in range(1,6)]:
    exp = experiment(importlib.import_module(f"experiment_configuration.{experiment_identifier}").config)
    ens = ensemble_GKLT(exp)

    ens.get_weights_uniqueness()
    out_folder = f"{data_dir}/{exp.initial_conditions_name}-x{exp.experiment_identifier[1]}/meta"
    os.makedirs(out_folder, exist_ok=True)
    xr.Dataset({'uniqueness':ens._weight_daily}).to_netcdf(f"{out_folder}/uniqueness_day_{exp.initial_conditions_name}-x{exp.experiment_identifier[1]}.nc")
    xr.Dataset({'uniqueness':ens._weight_stepwise}).to_netcdf(f"{out_folder}/uniqueness_step_{exp.initial_conditions_name}-x{exp.experiment_identifier[1]}.nc")
    xr.Dataset({'uniqueness':ens._weight}).to_netcdf(f"{out_folder}/uniqueness_season_{exp.initial_conditions_name}-x{exp.experiment_identifier[1]}.nc")

In [ ]:
# probabilities
for experiment_identifier in [f"c{i}" for i in range(1,6)] + [f"p{i}" for i in range(1,6)]:
    exp = experiment(importlib.import_module(f"experiment_configuration.{experiment_identifier}").config)
    ens = ensemble_GKLT(exp)
    ens._trajectories['obs'] = xr.open_mfdataset(f"{data_dir}/{exp.initial_conditions_name}-x{exp.experiment_identifier[1]}/day/atmos/tas-reg/*/*", concat_dim='sim', combine='nested')['tas'].load()
    ens.calc_scores_Ris_etc()

    out_folder = f"{data_dir}/{exp.initial_conditions_name}-x{exp.experiment_identifier[1]}/meta"
    os.makedirs(out_folder, exist_ok=True)
    xr.Dataset({'probability':ens._prob}).to_netcdf(f"{out_folder}/probability_season_{exp.initial_conditions_name}-x{exp.experiment_identifier[1]}.nc")